# Advanced Large Scale learning with ModelSelector
Very often we have many different products, regions, countries, shops...for which we need to delivery forecast. This can be easily done with `ModelSelector`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.model_selection import ModelSelector
from hcrystalball.utils import get_sales_data
from hcrystalball.wrappers import get_sklearn_wrapper
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## Get Dummy Data

In [ ]:
df = get_sales_data(n_dates=365*2, 
                    n_assortments=2, 
                    n_states=2, 
                    n_stores=2)#.assign(BelgiumHolidays="BE")

In [ ]:
df.head()

### Get predefined sklearn models, holidays and exogenous variables
Here for the sake of time, we will use the advantage of the ``create_gridsearch`` method for cv splits, default scorer etc. and just extend empty grid with two models

In [ ]:
ms = ModelSelector(
    frequency='D', 
    horizon=10, 
    country_code_column='HolidayCode',
#     country_code_column=['HolidayCode','BelgiumHolidays'])
)

In [ ]:
# see full default parameter grid in hands on exercise
ms.create_gridsearch(
    n_splits=2,
    between_split_lag=5, # create overlapping cv_splits
    sklearn_models=False,
    sklearn_models_optimize_for_horizon=False,
    autosarimax_models=False,
    prophet_models=False,
    tbats_models=False,
    exp_smooth_models=False,
    average_ensembles=False,
    stacking_ensembles=False,
    exog_cols=['Open','Promo','SchoolHoliday','Promo2'],
    holidays_days_before=2, 
    holidays_days_after=1, 
    holidays_bridge_days=True, 
)

In [ ]:
ms.add_model_to_gridsearch(get_sklearn_wrapper(LinearRegression, hcb_verbose=False))
ms.add_model_to_gridsearch(get_sklearn_wrapper(RandomForestRegressor, random_state=42, hcb_verbose=False))

### Run model selection with partitions
This can be done within classical for loop that enables you to see progress bar, or within parallelized prefect flow in case you would define `parallel_over_columns`, which must be subset of `partition_columns` and optionally add `executor` to point to your running `dask` cluster. Default uses `LocalExecutor`, you might also try `LocalDaskExecutor`, that prefect will spin up for you `DaskExecutor` if you have one already running and you want to connect to it.

In [ ]:
df

In [ ]:
# from prefect.engine.executors import LocalDaskExecutor
ms.select_model(df=df,
                target_col_name='Sales',
                partition_columns=['Assortment', 'State', 'Store'],               
#                 parallel_over_columns=['Assortment'],
#                 executor = LocalDaskExecutor(),
               )

In [ ]:
ms.get_partitions(as_dataframe=True)

In [ ]:
ms.plot_results(partitions=ms.partitions[:2], plot_from='2015-06');